# S2W6D5: Datasets 库实战 (Real-World Datasets)

今天 **Day 5**，我们要把战场转移到 **真实世界**。我们将使用 Hugging Face 经典的中文情感分析数据集 `ChnSentiCorp`，真正体验 **Apache Arrow** 带来的飞一般的速度，并掌握数据的**持久化存储**。

**🎯 核心目标**:

1.  **理论祛魅**: 理解为什么 Datasets 能在 8GB 内存的电脑上处理 100GB 数据 (Memory Mapping)。
2.  **实战演练**: 加载真实的 `ChnSentiCorp` 数据集，清洗、映射、保存。
3.  **工程效率**: 学会 `save_to_disk`，避免每次训练都重新预处理数据。

## 1 📚 理论：Apache Arrow 与内存映射

面试官常问：*“Python 的 List 和 Pandas 读取大文件都很慢且占内存，Hugging Face 是怎么解决的？”*

  * **传统方式 (Python Object)**: 每一个字符串、每一个数字都是一个对象，有巨大的 Overhead（头部开销）。读取 1GB 文本可能占 5GB 内存。
  * **HF Datasets 方式 (Apache Arrow)**:
      * **列式存储**: 数据像一根根紧凑的柱子排列，没有额外的 Python 对象开销。
      * **Memory Mapping (mmap)**:

![mmap](https://encrypted-tbn1.gstatic.com/licensed-image?q=tbn:ANd9GcTBNFnVT9w-t3IYzQlCp6OlrLYmLKEC_1o98KbprMpNm9Ls8I6m7QU6NbVlzFzNcdB7mjGrPL4i5NbG_dfsAV4BzJAv4XBFby-uB9mQ4VoPnsJNWq0)

操作系统直接把硬盘文件“映射”到虚拟内存地址。**你需要读哪一行，OS 才从硬盘加载哪一行**。这叫“零内存拷贝” (Zero-Copy)。
\* **结论**: 哪怕数据集有 1TB，只要你不一次性打印出来，它在 Python 里看起来就只占几 KB。

## 2 💻 代码实战：加载真实数据集

我们先配置镜像，尝试加载云端数据集。

### 2.1 加载数据 (Load from Hub)

In [4]:
import os
from datasets import load_dataset

# 1. 指向你存放文件的目录
# 假设你放在了这里
data_dir = "../../data/raw_data/ChnSentiCorp"

# 2. 定义文件路径字典
data_files = {
    "train": os.path.join(data_dir, "chn_senti_corp-train.arrow"),
    "validation": os.path.join(data_dir, "chn_senti_corp-validation.arrow"),
    "test": os.path.join(data_dir, "chn_senti_corp-test.arrow")
}

print("📂 正在从本地加载 Arrow 文件...")

try:
    # 3. 指定加载器为 "arrow"
    # 这会直接读取二进制文件，速度极快，无需解析脚本
    dataset = load_dataset("arrow", data_files=data_files)
    
    print("\n✅ 加载成功！")
    print(dataset)
    
    print("\n--- 查看第一条数据 ---")
    print(dataset['train'][0])
    
    # 提示：Arrow文件通常已经包含正确的列名，不需要改名
    # 如果列名还是不对，可以用 rename_column 自行调整
    
except Exception as e:
    print(f"❌ 加载失败: {e}")
    print("请检查路径是否正确，以及文件名是否与代码中一致。")

📂 正在从本地加载 Arrow 文件...


Generating train split: 9600 examples [00:00, 363149.75 examples/s]
Generating validation split: 1200 examples [00:00, 268006.65 examples/s]
Generating test split: 1200 examples [00:00, 246361.47 examples/s]


✅ 加载成功！
DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1200
    })
})

--- 查看第一条数据 ---
{'label': 1, 'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'}


### 2.2 数据清洗与过滤 (Filter)

真实数据往往是脏的。比如有些评论太短（"啊"），或者太长。我们需要清洗。

In [5]:
filtered_dataset = dataset.filter(lambda x: len(x['text']) > 5)

print(f"原始数量: {len(dataset['train'])}")
print(f"过滤后数量: {len(filtered_dataset['train'])}")

Filter: 100%|██████████| 1200/1200 [00:00<00:00, 89027.41 examples/s]

原始数量: 9600
过滤后数量: 9598


### 2.3 高效映射 (Map with Tokenizer)

这是今天的重头戏。我们要把清洗后的文本批量转为 ID。


In [7]:
from transformers import BertTokenizer

# 加载本地 Tokenizer
model_path = "../../data/pretrained_models/bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_path)

def preprocess_function(examples):
    # max_length=128: 情感分析通常不需要太长，截断能提升速度
    return tokenizer(
        examples["text"], 
        truncation=True, 
        max_length=128 
    )

# --- 核心：多进程批量映射 ---
# batched=True: 必须开！利用 Rust 底层加速
# num_proc=4: 开启 4 个进程并行处理 (如果你的 CPU 核够多)
tokenized_dataset = filtered_dataset.map(
    preprocess_function, 
    batched=True, 
    num_proc=4,
    remove_columns=["text"] # 处理完后，原始文本就不需要了，删掉省空间
)

print("\n--- 处理后的列名 ---")
print(tokenized_dataset['train'].column_names)
# 应该包含: input_ids, token_type_ids, attention_mask, label

Map (num_proc=4): 100%|██████████| 1200/1200 [00:00<00:00, 2230.37 examples/s]


--- 处理后的列名 ---
['label', 'input_ids', 'token_type_ids', 'attention_mask']


### 2.4 数据持久化 (Save to Disk)

**这是工业界最关键的一步**。
预处理（Tokenizer）是很耗时的。你肯定不想明天训练模型时，又重新跑一遍上面的代码。
我们可以把处理好的 Arrow 文件存到硬盘上。

In [ ]:
save_path = "../../data/processed_data/chn_senti_corp"

# 保存
tokenized_dataset.save_to_disk(save_path)
print(f"✅ 数据集已保存到: {save_path}")

# --- 模拟：下次训练时直接加载 ---
# from datasets import load_from_disk
# reloaded_dataset = load_from_disk(save_path)

Saving the dataset (1/1 shards): 100%|██████████| 1200/1200 [00:00<00:00, 104040.45 examples/s]

✅ 数据集已保存到: ../../data/processed_data/chn_senti_corp


## 3 💥 面试深挖 (Deep Dive)

**Q: `dataset.map` 和 Python 的 `for` 循环有什么本质区别？**

  * **A**:
    1.  **并行性**: `map` 支持 `batched=True` 和 `num_proc`，可以并行调用 Tokenizer (Rust 实现)，比单线程 Python 循环快 10-100 倍。
    2.  **内存管理**: `map` 处理后的结果会直接写入硬盘缓存 (Arrow 文件)，而不是堆积在 RAM 里，不会导致 OOM。

**Q: 为什么你说 Arrow 格式是“列式存储”，这有什么好处？**

  * **A**: 在深度学习中，我们通常需要取出一整个 Batch 的 `input_ids`（一列）传给 GPU。
      * **行式存储 (CSV/JSON)**: 必须读完每一行，才能把 `input_ids` 抠出来。
      * **列式存储 (Arrow)**: `input_ids` 在硬盘上是连续存放的，读取时只需要一次连续 I/O，速度极快。

## 4 ⚔️ 每日算法题 (LeetCode Daily)

今天这道题是面试中 **双指针 (Two Pointers)** 算法的巅峰之作，也是面试官最爱考的题目之一。

### 🎯 题目: [LeetCode 15. 三数之和 (3Sum)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium (接近 Hard)
  * **描述**: 给你一个整数数组 `nums`，判断是否存在三个元素 $a, b, c$ 使得 $a + b + c = 0$？请你找出所有和为 0 且不重复的三元组。
  * **核心思路**: **排序 + 双指针**。
    1.  先对数组进行排序。
    2.  固定第一个数 `nums[i]`。
    3.  用两个指针 `L` (左) 和 `R` (右) 在 `i` 后面的区间里寻找 `nums[L] + nums[R] = -nums[i]`。
    4.  **难点**: 如何去重？（比如 `[-1, -1, 2]`，两个 `-1` 会导致重复结果）。

-----

### ✅ Day 5 任务清单

1.  **运行代码**: 加载 `ChnSentiCorp`，跑通 `filter` -\> `map` -\> `save_to_disk` 的全流程。
      * *注：如果 `ChnSentiCorp` 下载慢，可以换成 `load_dataset("csv", data_files=...)` 加载我们昨天的模拟 CSV，流程是一模一样的。*
2.  **查看缓存**: 去 `../../data/processed_data/` 目录下看看生成了什么文件（应该会有 `dataset.arrow`）。
3.  **LeetCode**: 攻克 LeetCode 15。